# Project Group - 26

Members: Joris Veen, Frans Kraanen, Romee Hoogeveen, Jasper van Raaij, Olivier Klein Schiphorst

Student numbers: 4722868, 4727282, 4850025, 5177006, 4705106

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

To transport goods on the road vehicles need fuel to drive. The prices of fuel can fluctuate a lot so this can lead to high or low costs for tranport companies. This research will try to find out if the prices of fuel have an influence on the road transport. The following research question is proposed:


**Research question:** 

- How do fuel prices influence road transport of goods expressed in tons in the Netherlands?

**sub-questions:** 

- How do fuel prices influence incoming transport of goods in the Netherlands?
- How do fuel prices influence throughgoing transport of goods in the Netherlands?
- How do fuel prices influence outgoing transport of goods in the Netherlands?

First the available data will be explored and secondly we will focus to answer our research questions. The data available is from 2007 till 2022. The data used is focussed on the Netherlands only.

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

- CBS: International trade and transport in the Netherlands https://opendata.cbs.nl/statline/#/CBS/nl/dataset/84668NED/table?ts=1696403890264
- CBS: Fuel prices in the Netherlands https://opendata.cbs.nl/#/CBS/nl/dataset/80416ned/table

# Data Pipeline

Data Preparation

In [1]:
# import necassary libraries
import pandas as pd

#load datasets
fuel_data = pd.read_csv('fuel_prices.csv', delimiter=';')
transport_data = pd.read_csv('road_transport.csv', delimiter=';')

In [2]:
fuel_data.head() 

,Perioden,Benzine,Diesel,Lpg
0,2006 zondag 1 januari,"1,325","1,003","0,543"
1,2006 maandag 2 januari,"1,328","1,007","0,542"
2,2006 dinsdag 3 januari,"1,332","1,007","0,54"
3,2006 woensdag 4 januari,"1,348","1,02","0,55"
4,2006 donderdag 5 januari,"1,347","1,021","0,55"


In [3]:
transport_data.head()

,Goederenstromen,Perioden,1000 kg
0,Inkomend transport; totaal,2007*,103838966
1,Inkomend transport; totaal,2008*,103405705
2,Inkomend transport; totaal,2009*,89433416
3,Inkomend transport; totaal,2010*,89935769
4,Inkomend transport; totaal,2011*,91253760


In [4]:
#only run this cell one time
from datetime import datetime

# Custom function to convert a date string to a Pandas datetime
def parse_date(date_str):
    for i, month_name in enumerate(dutch_months):
        date_str = date_str.replace(month_name, datetime(2000, i+1, 1).strftime('%B'))

    for i, day_name in enumerate(dutch_days):
        date_str = date_str.replace(day_name, datetime(2000, 1, i+1).strftime('%A'))

    date_format = '%Y %A %d %B'
    return pd.to_datetime(date_str, format=date_format)

# Define Dutch months and day names
dutch_months = ['januari', 'februari', 'maart', 'april', 'mei', 'juni', 'juli', 'augustus', 'september', 'oktober', 'november', 'december']
dutch_days = ['zondag', 'maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag']

# Apply the custom function to the "Perioden" column
fuel_data['Perioden'] = fuel_data['Perioden'].apply(parse_date)

#set perioden to datetime format year
fuel_data['Perioden'] = fuel_data['Perioden'].dt.year

#replace , to . and set to float
fuel_data['Benzine'] = fuel_data['Benzine'].str.replace(',', '.', regex=False)
fuel_data['Benzine'] = fuel_data['Benzine'].astype(float)
fuel_data['Diesel'] = fuel_data['Diesel'].str.replace(',', '.', regex=False)
fuel_data['Diesel'] = fuel_data['Diesel'].astype(float)
fuel_data['Lpg'] = fuel_data['Lpg'].str.replace(',', '.', regex=False)
fuel_data['Lpg'] = fuel_data['Lpg'].astype(float)

# Group by year and calculate the mean for all columns
fuel_data = fuel_data.groupby('Perioden').mean().reset_index()

# Display the DataFrame with the converted dates
fuel_data.head()

,Perioden,Benzine,Diesel,Lpg
0,2006,1.373255,1.043211,0.520479
1,2007,1.414156,1.059721,0.538271
2,2008,1.476393,1.241314,0.598790
3,2009,1.354011,1.012929,0.509482
4,2010,1.503186,1.170838,0.644268


In [5]:
#only run this cell one time
#remove asterix from tansport data
transport_data['Perioden'] = transport_data['Perioden'].str.replace('*', '', regex=False)

#set perioden to datetime format year
transport_data['Perioden'] = pd.to_datetime(transport_data['Perioden'])
transport_data['Perioden'] = transport_data['Perioden'].dt.year

In [6]:
transport_data.head()

,Goederenstromen,Perioden,1000 kg
0,Inkomend transport; totaal,2007,103838966
1,Inkomend transport; totaal,2008,103405705
2,Inkomend transport; totaal,2009,89433416
3,Inkomend transport; totaal,2010,89935769
4,Inkomend transport; totaal,2011,91253760


In [7]:
### QUESTION 1
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## First dataset
#Percentage (%) differences between fuel prices with respect to the first year in the dataset (2006)
fuel_data['Benzine Price Diff (%)'] = ((fuel_data['Benzine'] - fuel_data['Benzine'].iloc[0]) / fuel_data['Benzine'].iloc[0]) * 100
fuel_data['Diesel Price Diff (%)'] = ((fuel_data['Diesel'] - fuel_data['Diesel'].iloc[0]) / fuel_data['Diesel'].iloc[0]) * 100
fuel_data['Lpg Price Diff (%)'] = ((fuel_data['Lpg'] - fuel_data['Lpg'].iloc[0]) / fuel_data['Lpg'].iloc[0]) * 100

## Second dataset
#filtering the data so only the value "inkomend transport; totaal" is given
transport_data_filtered = transport_data[(transport_data['Goederenstromen'] == 'Inkomend transport; totaal')]

#Percentage (%) differences between "Inkomend transport; totaal" with respect to the first year in the dataset (2007)
transport_data_filtered['Transport Inkomend Diff (%)'] = ((transport_data_filtered['1000 kg'] - transport_data_filtered['1000 kg'].iloc[0]) / transport_data_filtered['1000 kg'].iloc[0]) * 100

## Creating the plot
# Subplot for shared x-axis 
Figure = make_subplots(rows = 1, cols = 1, shared_xaxes = True,
                    subplot_titles = ['Fuel and Incoming Transport Differences'])

# Add annotations for first dataset, with three different columns
for i in ['Benzine Price Diff (%)', 'Diesel Price Diff (%)', 'Lpg Price Diff (%)']:
    Figure.add_trace(go.Scatter(x = fuel_data['Perioden'], 
                                y = fuel_data[i], 
                                mode = 'lines+text', 
                                name = i,
                                text = fuel_data[i].round(1).astype(str) + '%',
                                textposition = "top center"))

# Add annontations for second dataset
Figure.add_trace(go.Scatter(x = transport_data_filtered['Perioden'], 
                            y = transport_data_filtered['Transport Inkomend Diff (%)'], 
                            mode = 'lines+text', 
                            name = 'Transport Inkomend Diff (%)',
                            text = transport_data_filtered['Transport Inkomend Diff (%)'].round(1).astype(str) + '%',
                            textposition = "top center"))

# Update the layout of the figure + add title + set height and width for the figure
Figure.update_layout(title_text = 'Percentage differences over time with respect to the first year of the datasets',
                        xaxis_title = 'Year',
                        yaxis_title = 'Percentage Difference',
                        yaxis_range = [-50, 100],
                        height = 750,
                        width = 1500)

Figure.show()


C:\Users\olivi\AppData\Local\Temp\ipykernel_20552\4232926163.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transport_data_filtered['Transport Inkomend Diff (%)'] = ((transport_data_filtered['1000 kg'] - transport_data_filtered['1000 kg'].iloc[0]) / transport_data_filtered['1000 kg'].iloc[0]) * 100


In [8]:
### QUESTION 2a
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## First dataset
#Percentage (%) differences between fuel prices with respect to the first year in the dataset (2006)
fuel_data['Benzine Price Diff (%)'] = ((fuel_data['Benzine'] - fuel_data['Benzine'].iloc[0]) / fuel_data['Benzine'].iloc[0]) * 100
fuel_data['Diesel Price Diff (%)'] = ((fuel_data['Diesel'] - fuel_data['Diesel'].iloc[0]) / fuel_data['Diesel'].iloc[0]) * 100
fuel_data['Lpg Price Diff (%)'] = ((fuel_data['Lpg'] - fuel_data['Lpg'].iloc[0]) / fuel_data['Lpg'].iloc[0]) * 100

## Second dataset
#filtering the data so only the value "Inkomende doorvoer; totaal" is given
transport_data_filtered = transport_data[(transport_data['Goederenstromen'] == 'Inkomende doorvoer; totaal')]

#Percentage (%) differences between "Inkomende doorvoer; totaal" with respect to the first year in the dataset (2007)
transport_data_filtered['Inbound Transit Diff (%)'] = ((transport_data_filtered['1000 kg'] - transport_data_filtered['1000 kg'].iloc[0]) / transport_data_filtered['1000 kg'].iloc[0]) * 100

## Creating the plot
# Subplot for shared x-axis 
Figure = make_subplots(rows = 1, cols = 1, shared_xaxes = True,
                    subplot_titles = ['Fuel and Inbound Transit Differences'])

# Add annotations for first dataset, with three different columns
for i in ['Benzine Price Diff (%)', 'Diesel Price Diff (%)', 'Lpg Price Diff (%)']:
    Figure.add_trace(go.Scatter(x = fuel_data['Perioden'], 
                                y = fuel_data[i], 
                                mode = 'lines+text', 
                                name = i,
                                text = fuel_data[i].round(1).astype(str) + '%',
                                textposition = "top center"))

# Add annontations for second dataset
Figure.add_trace(go.Scatter(x = transport_data_filtered['Perioden'], 
                            y = transport_data_filtered['Inbound Transit Diff (%)'], 
                            mode = 'lines+text', 
                            name = 'Inbound Transit Diff (%)',
                            text = transport_data_filtered['Inbound Transit Diff (%)'].round(1).astype(str) + '%',
                            textposition = "top center"))

# Update the layout of the figure + add title + set height and width for the figure
Figure.update_layout(title_text = 'Percentage differences over time with respect to the first year of the datasets',
                        xaxis_title = 'Year',
                        yaxis_title = 'Percentage Difference',
                        yaxis_range = [-50, 100],
                        height = 750,
                        width = 1500)

Figure.show()

C:\Users\olivi\AppData\Local\Temp\ipykernel_20552\3096625976.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
### QUESTION 2b
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## First dataset
#Percentage (%) differences between fuel prices with respect to the first year in the dataset (2006)
fuel_data['Benzine Price Diff (%)'] = ((fuel_data['Benzine'] - fuel_data['Benzine'].iloc[0]) / fuel_data['Benzine'].iloc[0]) * 100
fuel_data['Diesel Price Diff (%)'] = ((fuel_data['Diesel'] - fuel_data['Diesel'].iloc[0]) / fuel_data['Diesel'].iloc[0]) * 100
fuel_data['Lpg Price Diff (%)'] = ((fuel_data['Lpg'] - fuel_data['Lpg'].iloc[0]) / fuel_data['Lpg'].iloc[0]) * 100

## Second dataset
#filtering the data so only the value "Uitgaande doorvoer; totaal" is given
transport_data_filtered = transport_data[(transport_data['Goederenstromen'] == 'Uitgaande doorvoer; totaal')]

#Percentage (%) differences between "Uitgaande doorvoer; totaal" with respect to the first year in the dataset (2007)
transport_data_filtered['Outbound Transit Diff (%)'] = ((transport_data_filtered['1000 kg'] - transport_data_filtered['1000 kg'].iloc[0]) / transport_data_filtered['1000 kg'].iloc[0]) * 100

## Creating the plot
# Subplot for shared x-axis 
Figure = make_subplots(rows = 1, cols = 1, shared_xaxes = True,
                    subplot_titles = ['Fuel and Outbound Transit Differences'])

# Add annotations for first dataset, with three different columns
for i in ['Benzine Price Diff (%)', 'Diesel Price Diff (%)', 'Lpg Price Diff (%)']:
    Figure.add_trace(go.Scatter(x = fuel_data['Perioden'], 
                                y = fuel_data[i], 
                                mode = 'lines+text', 
                                name = i,
                                text = fuel_data[i].round(1).astype(str) + '%',
                                textposition = "top center"))

# Add annontations for second dataset
Figure.add_trace(go.Scatter(x = transport_data_filtered['Perioden'], 
                            y = transport_data_filtered['Outbound Transit Diff (%)'], 
                            mode = 'lines+text', 
                            name = 'Outbound Transit Diff (%)',
                            text = transport_data_filtered['Outbound Transit Diff (%)'].round(1).astype(str) + '%',
                            textposition = "top center"))

# Update the layout of the figure + add title + set height and width for the figure
Figure.update_layout(title_text = 'Percentage differences over time with respect to the first year of the datasets',
                        xaxis_title = 'Year',
                        yaxis_title = 'Percentage Difference',
                        yaxis_range = [-50, 100],
                        height = 750,
                        width = 1500)

Figure.show()

C:\Users\olivi\AppData\Local\Temp\ipykernel_20552\1539126517.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
### QUESTION 3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## First dataset
#Percentage (%) differences between fuel prices with respect to the first year in the dataset (2006)
fuel_data['Benzine Price Diff (%)'] = ((fuel_data['Benzine'] - fuel_data['Benzine'].iloc[0]) / fuel_data['Benzine'].iloc[0]) * 100
fuel_data['Diesel Price Diff (%)'] = ((fuel_data['Diesel'] - fuel_data['Diesel'].iloc[0]) / fuel_data['Diesel'].iloc[0]) * 100
fuel_data['Lpg Price Diff (%)'] = ((fuel_data['Lpg'] - fuel_data['Lpg'].iloc[0]) / fuel_data['Lpg'].iloc[0]) * 100

## Second dataset
#filtering the data so only the value "Uitgaand transport; totaal" is given
transport_data_filtered = transport_data[(transport_data['Goederenstromen'] == 'Uitgaand transport; totaal')]

#Percentage (%) differences between "Uitgaand transport; totaal" with respect to the first year in the dataset (2007)
transport_data_filtered['Outgoing Transport Diff (%)'] = ((transport_data_filtered['1000 kg'] - transport_data_filtered['1000 kg'].iloc[0]) / transport_data_filtered['1000 kg'].iloc[0]) * 100

## Creating the plot
# Subplot for shared x-axis 
Figure = make_subplots(rows = 1, cols = 1, shared_xaxes = True,
                    subplot_titles = ['Fuel and Outgoing Transport Differences'])

# Add annotations for first dataset, with three different columns
for i in ['Benzine Price Diff (%)', 'Diesel Price Diff (%)', 'Lpg Price Diff (%)']:
    Figure.add_trace(go.Scatter(x = fuel_data['Perioden'], 
                                y = fuel_data[i], 
                                mode = 'lines+text', 
                                name = i,
                                text = fuel_data[i].round(1).astype(str) + '%',
                                textposition = "top center"))

# Add annontations for second dataset
Figure.add_trace(go.Scatter(x = transport_data_filtered['Perioden'], 
                            y = transport_data_filtered['Outgoing Transport Diff (%)'], 
                            mode = 'lines+text', 
                            name = 'Outgoing Transport Diff (%)',
                            text = transport_data_filtered['Outgoing Transport Diff (%)'].round(1).astype(str) + '%',
                            textposition = "top center"))

# Update the layout of the figure + add title + set height and width for the figure
Figure.update_layout(title_text = 'Percentage differences over time with respect to the first year of the datasets',
                        xaxis_title = 'Year',
                        yaxis_title = 'Percentage Difference',
                        yaxis_range = [-50, 100],
                        height = 750,
                        width = 1500)

Figure.show()

C:\Users\olivi\AppData\Local\Temp\ipykernel_20552\3157685575.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

